# 1. Data collection

In [170]:
import requests
import pandas as pd
import time
import os
from bs4 import BeautifulSoup
import time
import csv
import itertools
import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

## 1.1. Get the list of movies

In [158]:
big_page = 'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html'
big_page_response = requests.get(big_page)

In [159]:
big_page_soup = BeautifulSoup(big_page_response.text, 'html.parser')

## 1.2. Crawl Wikipedia

In [160]:
List_url=[]

In [ ]:
counter = 0
for i in big_page_soup.find_all('tr')[1:]:
    url = i.find_all('a')[0].get("href") # get the url in the second column of each row starting from the sec. row
    page_response= requests.get(url)  # reponse to the request
    
    if page_response.status_code == 200:  # 200 == means everthing is ok
        with open(os.path.join('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/html_files1',"movie"+str(counter+1)+".html"), "w") as file:
            page_soup = BeautifulSoup(page_response.text, 'html.parser')
            file.write(str(page_soup))  # writes the html page
            List_url.append(url) # add url to the list L
        counter += 1
    elif page_response.status_code == 429: # 429 == means that your doing too many requaets
        print('we must wait...')
        time.sleep(600) # wait 10 mins 
        page_soup = BeautifulSoup(page_response.text, 'html.parser')
        with open(os.path.join('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/html_files1',"movie"+str(counter+1)+".html"), "w") as file:
            file.write(str(page_soup))  # writes the html page
            List_url.append(url) # add url to the list L
        counter += 1
    elif counter%25==0:  # every 25 files, wait 10 seconds
        time.sleep(10)
        page_soup = BeautifulSoup(page_response.text, 'html.parser')
        with open(os.path.join('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/html_files1',"movie"+str(counter+1)+".html"), "w") as file:
            file.write(str(page_soup))  # writes the html page
            List_url.append(url) # add url to the list L
        counter += 1


    print("parsed movie " + str(counter))

In [63]:
with open('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/Downdloaded_url', 'w', newline="") as myfile:
     wr = csv.writer(myfile)
     wr.writerow(List_url)

## 1.3 Parse downloaded pages

In [171]:
lst = []
with open('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/Downdloaded_url', 'r') as f:
    reader = csv.reader(f)
    lst.append(list(reader)[0])
List_url = lst[0]

In [172]:
ps = PorterStemmer()
stop = set(stopwords.words('english'))
Vocabulary = {}

In [173]:
# questo mi derve per pulire gli output dell'infobox
def clean_col(string): 
    clean1 = string.strip('\n') # remove all escape characters
    pattern = r'\[.*?\]'  # remove all square brakets and content
    clean2 = re.sub(pattern, '', clean1)
    return(clean2)

# questo per ricavare i dati dall'infobox
def info_tsv(film_soup, movie_number, filename):
    
    pattern = r'\[.*?\]'  # remove all square brakets and content
    
    infb = ['Directed by', 'Produced by', 'Written by', 'Screenplay by',
           'Story by', 'Based on', 'Starring', 'Narrated by', 'Music by',
           'Cinematography', 'Edited by', 'Production company', 
            'Distributed by', 'Release date', 'Running time', 'Country',
           'Language', 'Budget'] # lista degli elementi che vogliamo
                                # ho aagiunto anche cose inutili sempre 
                                # per provare il codice
    L = []  # Questa lista conterrà tutti gli elementi trovati nell'infob.
            # e quelli mancanti. 
        
    # Retrieve the Title   
        
    Initial_Title = film_soup.select("#firstHeading")[0].text
    L.append(['Title', str(Initial_Title)])
    
    
    # Retrieve the Intro
    Intro2 = []
    Intro3 = []  # don't remove duplicates
    
    contents = film_soup.findAll('div', attrs={'id': 'toc'}) 
    if len(contents) > 0:
        Intr = contents[0].fetchPreviousSiblings('p')
        Intro = []
        if len(Intr) > 0:
            for p in reversed(Intr):
                if len(p.text) > 5:
                    Intro.append(p.text)
                else:
                    continue
            Intro = ''.join(Intro)
            Final_intro = re.sub(pattern, '', Intro)
            
            if len(Final_intro)>20:
                for small_word in Final_intro.split():
                    row_cl = small_word.strip()  #  strips all kinds of trailing whitespace by default
                    row_cl_noPunct = re.sub(r'[^\w\s]','',row_cl) # remove puntuation
                    clean_row = [i for i in row_cl_noPunct.lower().split() if i not in stop] #remove stopwords

                    #if after cleaning you reamin with  noting append an empty string
                    if clean_row is None or len(clean_row) == 0:
                            Intro2.append('')

                    # if after cleaning there are still some useful/important words, append them
                    else:
                        for word in clean_row:
                            Intro2.append(ps.stem(word))
                    Intro3 = Intro2
                    Intro2 = list(set(Intro2))
                L.append(['Intro', Final_intro.strip()])
            else:
                L.append(['Intro', 'NA'])
    
    
    # Retrieve the Plot
    
    plot = []
    FFinal_plot = []
    FFinal_plot2 = []   # don't remove duplicates
    possibles = ['Plot','Synopsis','Plot synopsis','Plot summary', 
                 'Story','Plotline','The Beginning','Summary',
                'Content','Premise', 'Intro', 'Intro']
    for i in possibles:
        # find the node with id of "Plot"
        items = film_soup.find(id=i)
        if items is None or len(items) == 0:
            continue
        else:
            # walk through the siblings of the parent (H2) node 
            # until we reach the next H2 node
            for element in items.parent.nextSiblingGenerator():
                if element.name == "h2":
                    break
                if hasattr(element, "text"):
                    plot.append(element.text)
    plot = "".join(plot)
    # remove all square brakets and content
    Final_plot = re.sub(pattern, '', plot)
    if len(Final_plot) > 20:
        for small_word in Final_plot.split():
            row_cl = Final_plot.strip()  #  strips all kinds of trailing whitespace by default
            row_cl_noPunct = re.sub(r'[^\w\s]','',row_cl) # remove puntuation
            clean_row = [i for i in row_cl_noPunct.lower().split() if i not in stop] #remove stopwords

            #if after cleaning you reamin with  noting append an empty string
            if clean_row is None or len(clean_row) == 0:
                    FFinal_plot.append('')

            # if after cleaning there are still some useful/important words, append them
            else:
                for word in clean_row:
                    FFinal_plot.append(ps.stem(word))
                FFinal_plot2 = FFinal_plot
                FFinal_plot = list(set(FFinal_plot))
        L.append(['Plot', Final_plot.strip()])
    else:
        L.append(['Plot', 'NA'])
        
    # merge plot anf Intro to create dict
    
    merged2 = Intro3+FFinal_plot
    
    
    L.append(['Merged', merged2])
    
    merged = list(set(FFinal_plot + Intro2))
    page_name = filename.replace("html", "tsv") # without extension
    for i in merged:
        if i not in Vocabulary.keys():
            Vocabulary[i] = [page_name]
        elif page_name not in Vocabulary[i]:
            Vocabulary[i].append(page_name)
    
    # Retrieve info in the infobox
        
    if len(film_soup.findAll('table', {'class': 'infobox vevent'}))>0:
        if len(film_soup.findAll('table', {'class': 'infobox vevent'})[0].tbody.findAll('tr'))>0:
            Title = film_soup.findAll('table', {'class': 'infobox vevent'})[0].tbody.findAll('tr')[0].text
            if len(Title)>75:
                
                L.append(['Name', 'NA'])
            else:
                L.append(['Name', Title]) # mi ricavo il titolo dall'infobox
            
        else:
            L.append(['Name', 'NA'])
            
    in_table = [] # mi savo gli elementi dell'infobox qui
    
    if len(film_soup.findAll('table', {'class': 'infobox vevent'}))>0:
        if len(film_soup.findAll('table', {'class': 'infobox vevent'})[0].tbody.findAll('tr'))>0:
            for row in film_soup.findAll('table', {'class': 'infobox vevent'})[0].tbody.findAll('tr')[1:]:
                first_col = row.findAll('th') # first column of infobox
                if len(first_col)>0:
                    second_col = row.findAll('td')
                    if len(second_col)>0:
                        second_col = row.findAll('td')[0].text # second column of info
                        new_second_col = clean_col(second_col) # clean the output

                        in_table.append(first_col[0].text) 
                        in_table.append(new_second_col)
                        

            # put NA to all missing values
            for i in infb:
                if i in in_table:
                    index = in_table.index(i)
                    L.append([i, in_table[index +1]])
                else:
                    L.append([i, 'NA']) 
    L.append(['Url', List_url[int(movie_number) -1]])
    return(L)

In [174]:
# TEST CODE

page = open("/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/html_files/movie9245.html", "r")
page = page.read()
f =  BeautifulSoup(page)

t = info_tsv(f,5, 'movie9245')
with open('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/tsv_files/'+ 'movie9245' +'.tsv', 'wt') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        for col in info_tsv(f,5, 'movie9245'):
                tsv_writer.writerow([col[0], col[1]])
t

[['Title', 'One Summer Love'],
 ['Intro',
  'One Summer Love, originally titled Dragonfly, is a 1976 romantic drama film directed by Gilbert Cates from a screenplay by N. Richard Nash. It stars Beau Bridges and Susan Sarandon and features Mildred Dunnock and Ann Wedgeworth.'],
 ['Plot',
  'After being released from a mental hospital, Jesse (Bridges) sets out to find and rejoin his off-beat family. While doing so, he meets a pretty young woman named Chloe (Sarandon) who works in a movie theatre, and they fall in love, which resolves his psychological problems.'],
 ['Merged',
  ['',
   'one',
   'direct',
   'susan',
   'romant',
   'love',
   'bridg',
   'drama',
   'film',
   'dragonfli',
   'richard',
   'beau',
   'cate',
   'titl',
   'mildr',
   'summer',
   '1976',
   'dunnock',
   'screenplay',
   'gilbert',
   'ann',
   'n',
   'star',
   'nash',
   'origin',
   'sarandon',
   'featur',
   'wedgeworth',
   'theatr',
   'meet',
   'jess',
   'releas',
   'love',
   'name',
   'br

### Creation of the tsv files

In [118]:
movies_done = []

In [119]:
paths = '/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/html_files/'

    
for filename in os.listdir(paths):
    if filename not in movies_done:
        if filename.endswith(".html"):
            ff = open(paths + filename, 'r')
            html = ff.read()
            film = BeautifulSoup(html)

            page_name = filename.rstrip('.html') # without extension
            movies_done.append(filename)
            
            movie_number = re.findall('\d+', filename )
            movie_number = movie_number[0]
            
            with open('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/tsv_files/'+page_name+'.tsv', 'wt') as out_file:
                tsv_writer = csv.writer(out_file, delimiter='\t')
                if info_tsv(film,movie_number , filename) is None or len(info_tsv(film, movie_number, filename)) == 0:
                    continue
                else:
                    for col in info_tsv(film, movie_number, filename):
                        tsv_writer.writerow([col[0], col[1]])
            print('tsv created '+ str(len(movies_done)))

            continue

        else:
            continue
    else:
        continue
print('DONE')



tsv created 1
tsv created 2
tsv created 3
tsv created 4
tsv created 5
tsv created 6
tsv created 7
tsv created 8
tsv created 9
tsv created 10
tsv created 11
tsv created 12
tsv created 13
tsv created 14
tsv created 15
tsv created 16
tsv created 17
tsv created 18
tsv created 19
tsv created 20
tsv created 21
tsv created 22
tsv created 23
tsv created 24
tsv created 25
tsv created 26
tsv created 27
tsv created 28
tsv created 29
tsv created 30
tsv created 31
tsv created 32
tsv created 33
tsv created 34
tsv created 35
tsv created 36
tsv created 37
tsv created 38
tsv created 39
tsv created 40
tsv created 41
tsv created 42
tsv created 43
tsv created 44
tsv created 45
tsv created 46
tsv created 47
tsv created 48
tsv created 49
tsv created 50
tsv created 51
tsv created 52
tsv created 53
tsv created 54
tsv created 55
tsv created 56
tsv created 57
tsv created 58
tsv created 59
tsv created 60
tsv created 61
tsv created 62
tsv created 63
tsv created 64
tsv created 65
tsv created 66
tsv created 67
tsv 

KeyboardInterrupt: 

In [ ]:
with open('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/Vocabulary.json', 'w') as outfile:
    json.dump(Vocabulary, outfile)